In [1]:
import os
import re
import requests
import time
import pandas as pd
from json import loads
from functools import reduce
from waybackpy import WaybackMachineCDXServerAPI
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm

In [2]:
SCRAPE_URLS = False
SCRAPE_JSONP = False

In [3]:
if SCRAPE_URLS:
    if not os.path.exists("snapshot_urls.txt"):
        url = "https://store.steampowered.com/stats/content/"
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/145.0.0.0 Safari/537.36"
        cdx = WaybackMachineCDXServerAPI(url, user_agent, start_timestamp="20080101000000", end_timestamp="20270101000000")
        snapshot_urls = []
        for item in cdx.snapshots():
            snapshot_urls.append(item.archive_url)

        with open("snapshot_urls.txt", "w") as f:
            for url in snapshot_urls:
                f.write(url + "\n")
    
    # Extract JSONP URLs from snapshot pages
    bad_urls = []
    if not os.path.exists("jsonp_urls.txt"):
        with open("snapshot_urls.txt", "r") as f:
            snapshot_urls = f.read().splitlines()

        jsonp_urls = []
        for url in tqdm(snapshot_urls, desc="Processing snapshot URLs"):
            try:
                ua = UserAgent()
                headers = {"User-Agent": ua.random}
                response = requests.get(url, headers=headers, timeout=300)
            except requests.RequestException as e:
                print(f"Error fetching URL {url}: {e}")
                bad_urls.append(url)
                continue
            soup = BeautifulSoup(response.content, "html.parser")
            for script in soup.find_all("script"):
                if script.string and "contentserver_bandwidth_stacked.jsonp" in script.string:
                    match = re.search(r'"(https?://[^"]*contentserver_bandwidth_stacked\.jsonp[^"]*)"', script.string)
                    if match:
                        jsonp_urls.append(match.group(1))
            time.sleep(10)  # Sleep to avoid overwhelming the server

        with open("jsonp_urls.txt", "w") as f:
            for url in jsonp_urls:
                f.write(url + "\n")
        
        if bad_urls:
            with open("bad_urls.txt", "w") as f:
                for url in bad_urls:
                    f.write(url + "\n")
        
    # Reprocess bad URLs
    with open("bad_urls.txt", "r") as f:
        bad_urls = f.read().splitlines()

    jsonp_urls = []
    for url in tqdm(bad_urls, desc="Reprocessing bad URLs"):
        try:
            ua = UserAgent()
            headers = {"User-Agent": ua.random}
            response = requests.get(url, headers=headers, timeout=300)
        except requests.RequestException as e:
            print(f"Error fetching URL {url}: {e}")
            continue
        soup = BeautifulSoup(response.content, "html.parser")
        for script in soup.find_all("script"):
            if script.string and "contentserver_bandwidth_stacked.jsonp" in script.string:
                match = re.search(r'"(https?://[^"]*contentserver_bandwidth_stacked\.jsonp[^"]*)"', script.string)
                if match:
                    jsonp_urls.append(match.group(1))
        time.sleep(10)  # Sleep to avoid overwhelming the server

    with open("jsonp_urls.txt", "a") as f:
        for url in jsonp_urls:
            f.write(url + "\n")

In [4]:
if SCRAPE_JSONP:
    with open("jsonp_urls.txt", "r") as f:
        urls = f.read().splitlines()

    all_dfs = []

    bad_urls = []
    for url in tqdm(urls):
        try:
            ua = UserAgent()
            headers = {"User-Agent": ua.random}
            response = requests.get(url, headers=headers, timeout=300)
        except requests.RequestException as e:
            print(f"Error fetching URL {url}: {e}")
            bad_urls.append(url)
            continue
        startidx = response.text.find("(")
        endidx = response.text.find(")")

        try:
            data = loads(response.text[startidx + 1 : endidx])
        except Exception as e:
            print(f"Error parsing JSONP from URL {url}: {e}")
            continue

        if "json" not in data:
            continue

        series_list = loads(data["json"])

        df_list = []
        for series in series_list:
            df_dict = {}
            region = series["label"]
            df_dict["Timestamp"] = [
                pd.to_datetime(x[0], unit="ms") for x in series["data"]
            ]
            df_dict[region] = [int(x[1]) for x in series["data"]]
            df_list.append(pd.DataFrame(df_dict))

        df = reduce(
            lambda x, y: pd.merge(x, y, on="Timestamp", how="outer"), df_list
        )
        df = df.sort_values("Timestamp").reset_index(drop=True)
        all_dfs.append(df)
        time.sleep(10)
    
    for url in bad_urls:
        try:
            ua = UserAgent()
            headers = {"User-Agent": ua.random}
            response = requests.get(url, headers=headers, timeout=300)
        except requests.RequestException as e:
            print(f"Error fetching URL {url}: {e}")
            continue
        startidx = response.text.find("(")
        endidx = response.text.find(")")

        try:
            data = loads(response.text[startidx + 1 : endidx])
        except Exception as e:
            print(f"Error parsing JSONP from URL {url}: {e}")
            continue

        if "json" not in data:
            continue

        series_list = loads(data["json"])

        df_list = []
        for series in series_list:
            df_dict = {}
            region = series["label"]
            df_dict["Timestamp"] = [
                pd.to_datetime(x[0], unit="ms") for x in series["data"]
            ]
            df_dict[region] = [int(x[1]) for x in series["data"]]
            df_list.append(pd.DataFrame(df_dict))

        df = reduce(
            lambda x, y: pd.merge(x, y, on="Timestamp", how="outer"), df_list
        )
        df = df.sort_values("Timestamp").reset_index(drop=True)
        all_dfs.append(df)
        time.sleep(10)

    df_combined = pd.concat(all_dfs, axis=0, ignore_index=True)
    df_combined.to_csv("old_data.csv", index=False)

In [10]:
current_df = pd.read_csv("../data/bandwidths.csv", parse_dates=["Timestamp"])
current_df

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2016-10-03 10:20:00,2,5.0,29.0,55.0,31.0,153.0,386.0,465.0,142.0
1,2016-10-03 10:30:00,2,5.0,30.0,53.0,31.0,154.0,391.0,473.0,139.0
2,2016-10-03 10:40:00,2,5.0,31.0,53.0,32.0,154.0,392.0,484.0,134.0
3,2016-10-03 10:50:00,2,5.0,31.0,51.0,32.0,158.0,390.0,498.0,134.0
4,2016-10-03 11:00:00,2,5.0,32.0,51.0,33.0,167.0,402.0,511.0,133.0
...,...,...,...,...,...,...,...,...,...,...
150380,2026-03-01 18:40:00,156,193.0,1373.0,134.0,3642.0,2933.0,5196.0,11957.0,8303.0
150381,2026-03-01 18:50:00,161,193.0,1360.0,132.0,3643.0,2881.0,4936.0,11918.0,8447.0
150382,2026-03-01 19:00:00,163,183.0,1351.0,130.0,3643.0,2805.0,4721.0,11828.0,8655.0
150383,2026-03-01 19:10:00,163,178.0,1351.0,132.0,3666.0,2721.0,4522.0,11740.0,8833.0


In [12]:
old_df = (
    pd.read_csv("old_data.csv", parse_dates=["Timestamp"])
    .sort_values("Timestamp")
    .reset_index(drop=True)
)
old_df = old_df.loc[
    (old_df["Timestamp"].dt.minute % 10 == 0)
    & (old_df["Timestamp"].dt.second == 0)
]
old_df["nan_count"] = old_df.isnull().sum(axis=1)
old_df = (
    old_df
    .sort_values(by=["Timestamp", "nan_count"])
    .drop_duplicates(subset="Timestamp", keep="first")
    .reset_index(drop=True)
)
old_df = old_df.drop(columns=["nan_count"])

# Filter to values not in current_df
old_df = old_df.loc[~old_df["Timestamp"].isin(current_df["Timestamp"])]
old_df

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
11,2016-10-29 06:20:00,5.0,7.0,35.0,107.0,84.0,213.0,684,346,717.0
12,2016-10-29 06:30:00,5.0,7.0,36.0,107.0,82.0,226.0,688,375,700.0
13,2016-10-29 06:40:00,5.0,8.0,37.0,109.0,80.0,235.0,688,405,666.0
14,2016-10-29 06:50:00,5.0,8.0,39.0,107.0,77.0,244.0,682,440,636.0
15,2016-10-29 07:00:00,5.0,8.0,40.0,107.0,74.0,253.0,682,474,617.0
...,...,...,...,...,...,...,...,...,...,...
37357,2024-06-12 16:00:00,55.0,125.0,906.0,216.0,1069.0,1991.0,9572,7479,3533.0
37358,2024-06-12 16:10:00,55.0,122.0,906.0,208.0,1076.0,1985.0,9177,7553,3630.0
37359,2024-06-12 16:20:00,57.0,123.0,911.0,204.0,1096.0,1974.0,8742,7564,3741.0
37360,2024-06-12 16:30:00,57.0,122.0,911.0,194.0,1113.0,1958.0,8579,7676,3879.0


In [15]:
combined = pd.concat([old_df, current_df], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)
regions_all = [
    "Central America",
    "Africa",
    "Middle East",
    "Oceania",
    "South America",
    "Russia",
    "Asia",
    "Europe",
    "North America",
]
combined[regions_all] = combined[regions_all].astype("Int64")
combined.to_csv("../data/bandwidths.csv", index=False)